### HDF5 Stuff

In [ ]:
import h5py
geant4_name = "improved_200cells_FPCD.hdf5"
f = h5py.File(h5_filename, 'r')
f.keys()
chunk_size=2000

print("[E, X, Y, Z]")
for i in range(5):
    print(f['hcal_cells'][1,i,:-1]) #XYZ, skip mask

[E, X, Y, Z]
[ 2.1972656e-04 -1.5000000e+03  1.0000000e+02  4.1257002e+03]
[ 2.746582e-04 -1.300000e+03  0.000000e+00  3.915100e+03]
[ 8.544922e-05 -1.200000e+03  1.000000e+02  3.961900e+03]
[ 7.93457e-05 -1.20000e+03  1.00000e+02  3.91510e+03]
[ 9.765625e-05 -1.300000e+03  2.000000e+02  3.868300e+03]


In [ ]:
h5_indexE = 0
h5_indexX = 1
h5_indexY = 2
h5_indexZ = 3

143


In [ ]:
import numpy as np
print(np.shape(f['hcal_cells']))

(1793700, 200, 5)


### Detector Parameters XYZ

In [13]:
voxel_factor=4

minXY = -2700
maxXY = 2700
widthXY = 100
ncellsXY = int(((maxXY-minXY)/widthXY)/voxel_factor)
binningXY = np.linspace(minXY,maxXY,ncellsXY+1)

minZ = 3820
maxZ = 4991
widthZ = 23.4
ncellsZ = int((((maxZ-minZ)/widthZ)+3)/voxel_factor)
binningZ = np.linspace(minZ,maxZ,ncellsZ+1)
print(ncellsZ)

n_truth_features = 2

13


## CaloScore takes [N,Z,X,Y] images. Our HDF5 files have [N,X,Y,Z]. Swap Z with X, and then X with Y

In [5]:
data = f['hcal_cells'][:chunk_size]
print("original:")
data[0,0,2] = 7.777777 #To guide the eye
print(data[0,:10])
temp_X = data[:,:,h5_indexX]
data[:,:,[1,3]] = data[:,:,[3,1]]#Swap Z and X
data[:,:,[2,3]] = data[:,:,[3,2]]#Swap Y and X
print("swapped")
print(data[0,:10])

original:
[[4.1503907e-04 1.0000000e+03 7.7777772e+00 3.8215000e+03 1.0000000e+00]
 [6.7138672e-04 1.0000000e+03 3.0000000e+02 3.8448999e+03 1.0000000e+00]
 [4.8217774e-04 1.0000000e+03 3.0000000e+02 3.8683000e+03 1.0000000e+00]
 [4.5776367e-04 1.1000000e+03 3.0000000e+02 3.8917000e+03 1.0000000e+00]
 [7.8735349e-04 1.1000000e+03 3.0000000e+02 3.9151001e+03 1.0000000e+00]
 [7.6904299e-04 1.1000000e+03 3.0000000e+02 3.9385000e+03 1.0000000e+00]
 [3.0517578e-04 1.1000000e+03 3.0000000e+02 3.9618999e+03 1.0000000e+00]
 [4.4555665e-04 1.1000000e+03 3.0000000e+02 3.9853000e+03 1.0000000e+00]
 [4.0893554e-04 1.1000000e+03 3.0000000e+02 4.0087000e+03 1.0000000e+00]
 [3.8452148e-03 1.1000000e+03 3.0000000e+02 4.0321001e+03 1.0000000e+00]]
swapped
[[4.1503907e-04 3.8215000e+03 1.0000000e+03 7.7777772e+00 1.0000000e+00]
 [6.7138672e-04 3.8448999e+03 1.0000000e+03 3.0000000e+02 1.0000000e+00]
 [4.8217774e-04 3.8683000e+03 1.0000000e+03 3.0000000e+02 1.0000000e+00]
 [4.5776367e-04 3.8917000e+03 1.

In [11]:
truth = f['cluster'][start:start+chunk_size,0:2]
print(truth[:10])

[[ 17.737766   16.070114 ]
 [  1.2629831  18.429943 ]
 [  2.1527076  17.271072 ]
 [ 11.540043   16.895714 ]
 [123.45066    16.294994 ]
 [  7.684786   18.410349 ]
 [ 16.285973   16.580107 ]
 [  2.877014   15.814756 ]
 [ 21.49815    18.268    ]
 [  2.5778146  15.683742 ]]


In [16]:
from tqdm import tqdm
nevents = np.shape(f['hcal_cells'])[0]
with h5py.File(f'epic_hcal_images_{voxel_factor}x{voxel_factor}.h5', 'w') as newfile:
    # create empty data set
    dset = newfile.create_dataset('calo_images', shape=(nevents, ncellsXY, ncellsXY, ncellsZ),
                            maxshape=(None, ncellsXY, ncellsXY, ncellsZ), chunks=(chunk_size, ncellsXY, ncellsXY, ncellsZ),
                            dtype=np.float32)
    truth_dset = newfile.create_dataset('truth_features', shape=(nevents, n_truth_features),
                            maxshape=(None, n_truth_features), chunks=(chunk_size, n_truth_features),
                            dtype=np.float32)
    
    for chunk in tqdm(range(int(nevents/chunk_size))):
    #for chunk in range(10):
        # add chunk of rows
        start = chunk*chunk_size
        #print(start)
        data = f['hcal_cells'][start:start+chunk_size]
        data[:,:,[1,3]] = data[:,:,[3,1]]#Z-Data is now at X-index, and X-Data is at previous Z-index
        data[:,:,[2,3]] = data[:,:,[3,2]]
        
        truth = f['cluster'][start:start+chunk_size,0:2] #genP, genTheta
        
        images = []
        for evt in range(chunk_size):
            counts, binedges = np.histogramdd(data[evt,:,:-2], bins=(ncellsXY,ncellsXY,ncellsZ), weights=data[evt,:,h5_indexE])
            images.append(counts)
        images=np.asarray(images)
        
        dset[start:start+chunk_size, :, :,:] = images[:, :, :,:]
        truth_dset[start:start+chunk_size, :] = truth[:,:]

100%|██████████| 897/897 [09:15<00:00,  1.61it/s]


In [7]:
with h5py.File('epic_hcal_images.h5', 'r') as newfile:
    #print(f['calo_images'].attrs['last_index'])
    print(newfile['calo_images'].shape)
    #print(f['embeds'][0:26])
    #print(newfile['calo_images'][0])
    for i in range(13):
        for row in newfile['calo_images'][0,:,:,i]:
            print(row)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'epic_hcal_images.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)